# This notebook is for creating useful data frame from raw data


In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# getting the total data in total_story_data.csv
files = os.listdir('Data')
f=open('total_story_data.csv','w')
for file in files:
    path=os.path.join('Data',file)
    g=open(path,'r')
    data=g.read()
    data=data.replace(',',' ')
    g.close()
    f.write(data)
    f.write('\n')
f.close()
    

In [3]:
# Creating a dataframe
df=pd.read_csv('total_story_data.csv',names=['story'],encoding='ANSI')

In [4]:
pd.set_option("display.max_colwidth",100)

In [5]:
df.head(2)

,story
0,ï»¿As a Data user I want to have the 12-19-2017 deletions processed.
1,As a UI designer I want to redesign the Resources page so that it matches the new Broker desig...


In [6]:
df['story'][0]=df['story'][0][3:]

In [7]:
df.head(3)

,story
0,As a Data user I want to have the 12-19-2017 deletions processed.
1,As a UI designer I want to redesign the Resources page so that it matches the new Broker desig...
2,As a UI designer I want to report to the Agencies about user testing so that they are aware of...


In [8]:
df[df.duplicated()==True]

,story
10,As a UI designer I want to move on to round 2 of Homepage edits so that I can get approvals fr...
29,As an Agency user I want to be able to include a large number of flexfields without performance...
61,As a user I want to access the raw agency published files from FABS via USAspending.
143,As an Applicant I want to Complete Building Development Project so that I can receive any post...
549,As a camp worker I want to be able to see which campers I am in charge of and where so that I ...
1616,As a data curator I want to upload a new version of an item that has changed while still allow...


In [9]:
df=df.drop_duplicates()

In [10]:
df=df.reset_index()

In [11]:
df.columns

Index(['index', 'story'], dtype='object')

In [12]:
df=df.drop('index',axis=1)

In [13]:
df.head(2)

,story
0,As a Data user I want to have the 12-19-2017 deletions processed.
1,As a UI designer I want to redesign the Resources page so that it matches the new Broker desig...


In [14]:
# Creating a seperate column for user role
import re
def get_role(text):
    pattern = re.compile(r'As (the|an|a)(\s+[\w\/ ]+)(  I)')
    extract=pattern.findall(text)
    try:
        return extract[0][1].strip()
    except:
        return 'No match'
    
    

In [15]:
res=get_role(df['story'][8])

In [16]:
res

'DevOps engineer'

In [17]:
# Making a new column for role
df['role']=df['story'].apply(get_role)

In [18]:
df.head(2)

,story,role
0,As a Data user I want to have the 12-19-2017 deletions processed.,Data user
1,As a UI designer I want to redesign the Resources page so that it matches the new Broker desig...,UI designer


In [19]:
# cheking for tabs
df[df['story'].str.find('\t')!=-1].shape

(55, 2)

In [20]:
# removing tabs
df['story']=df['story'].apply(lambda x:x.replace('\t',' '))

In [21]:
df.head(2)

,story,role
0,As a Data user I want to have the 12-19-2017 deletions processed.,Data user
1,As a UI designer I want to redesign the Resources page so that it matches the new Broker desig...,UI designer


In [22]:
# cheking for tabs
df[df['story'].str.find('\t')!=-1].shape

(0, 2)

In [23]:
df['role'].nunique()

226

In [24]:
df[df['role'].str.split().str.len()>5]

,story,role
291,As a ResearcherPublisher I want an integration with Zenodo so that when I store my dataset in ...,ResearcherPublisher I want an integration with Zenodo so that when I store my dataset in GitHub
295,As a Researcher working with data in NetCDF I want NetCDF integration so that I can store my ...,Researcher working with data in NetCDF
374,As a site member who has read a teaser on the front page I want to read the entire article so t...,site member who has read a teaser on the front page
463,As an NSF person interested in attending user interviews I want to know when they are scheduled...,NSF person interested in attending user interviews
1096,As a authenticateduser I want to edit my session so that if I make a mistake I can correct it.,authenticateduser I want to edit my session so that if I make a mistake
1468,As a researcher I want to access digitized archival materials without mediation so that I can ...,researcher I want to access digitized archival materials without mediation so that I can decid...
1570,As an Extension Administrator or Staff Content Manager I want to get periodic IR reminders inc...,Extension Administrator or Staff Content Manager
1594,As an asset manager I want to be assured that the digital copy I deposit is stored redundantly ...,asset manager I want to be assured that the digital copy I deposit is stored redundantly so th...


In [25]:
df['role'][291]='ResearcherPublisher'
df['role'][295]='Researcher'
df['role'][1096]='authenticateduser'
df['role'][1468]='Researcher'
df['role'][1594]='manager'

In [26]:
df[df['role']=='No match']

,story,role
294,As a ResearcherDeveloper would like the ability import/export from MS-SQL so that I can use Da...,No match
702,As repository manager I want to view authenticated active users and anonymous users.,No match
730,As a non-Duke researcher I want to describe my team and mine's objects in the repository after ...,No match
830,As lab administrator I want to have increased access to the system to support regular users.,No match
1276,AS an externalcoordinator I want to access data from Bath collaborators off campus so that I c...,No match
1314,As repository manager I want to know all the files belonging to an object.,No match


In [27]:
#Manually correcting the roles:
df['role'][294]="ResearcherDeveloper"
df['role'][702]="Repository manager"
df['role'][730]="Researcher"
df['role'][830]="administrator"
df['role'][1276]="externalcoordinator"
df['role'][1314]="Repository manager"


In [28]:
# Made a dictionary for assigning similar roles Using existing roles data withnthehelp of excel filtering

role_dict={"Data user" :"user",
"UI designer" :"designer",
"Developer" :"developer",
"Developer" :"developer",
"DevOps engineer" :"engineer",
"UI designer" :"designer",
"Broker user" :"user",
"Website user" :"user",
"owner" :"owner",
"user" :"user",
"Agency user" :"user",
"Owner" :"owner",
"data user" :"user",
"agency user" :"user",
"website user" :"user",
"broker team member" :"member",
"broker user" :"user",
"FABS user" :"user",
"tester" :"tester",
"Public User" :"user",
"ProspectiveApplicant" :"applicant",
"Applicant" :"applicant",
"Customer" :"Customer",
"Staff member" :"staff",
"staff member" :"staff",
"Planning Staff member" :"staff",
"Zoning Staff member" :"staff",
"Plan Review Staff member" :"staff",
"Plan Review Staff Supervisor" :"staff",
"Plan Review Staff Member" :"staff",
"Development Staff member" :"staff",
"Inspection Staff member" :"staff",
"Inspection Staff Supervisor" :"staff",
"Inspection Supervisor" :"supervisor",
"Inspector" :"inspector",
"Enforcement Staff member" :"staff",
"l Staff member" :"staff",
"County Staff Member" :"staff",
"Technical Staff member" :"staff",
"admin" :"admin",
"executive" :"executive",
"employee" :"employee",
"employee from the HR department" :"employee",
"developer" :"developer",
"company" :"company",
"company accountant" :"company",
"superuser" :"user",
"recyclingfacility representative" :"representative",
"recyclingfacility" :"representative",
"Data Publishing User" :"user",
"Platform Administrator" :"admin",
"Data Consuming User" :"user",
"API User" :"user",
"OpenSpending Community Member" :"member",
"User" :"user",
"API user" :"user",
"Platform administrator" :"admin",
"Researcher" :"researcher",
"Publisher" :"publisher",
"ResearcherPublisher" :"researcher",
"Machine Learning expert" :"analyst",
"ResearcherGovernment Publisher" :"researcher",
"Researcher  working with data" :"analyst",
"ResearcherDeveloper" :"developer",
"Researcher  working with data in NetCDF" :"analyst",
"Repository Manager" :"manager",
"publisher" :"publisher",
"DeveloperWrangler" :"developer",
"government Publisher" :"publisher",
"Civic Tech Activist" :"activist",
"DeveloperDataWrangler" :"developer",
"DeveloperPublisher" :"developer",
"RepositoryManagerResearcher" :"manager",
"site member" :"member",
"Practitioner" :"analyst",
"Trainer" :"trainer",
"trainer" :"trainer",
"site administrator" :"admin",
"site visitor" :"visitor",
"site editor" :"editor",
"site admin" :"admin",
"visitor" :"visitor",
"recruiter" :"recruiter",
"advertiser" :"advertiser",
"site member who has read a teaser on the front page" :"member",
"CSM" :"manager",
"corporatesponsor" :"sponsor",
"CST" :"tester",
"member" :"member",
"team member" :"member",
"stakeholder" :"stakeholder",
"NSF employee" :"employee",
"designer" :"designer",
"user researcher" :"user",
"content editor" :"editor",
"research participant" :"researcher",
"NSF administrator" :"admin",
"researcher" :"researcher",
"CMS administrator" :"admin",
"CMS SME" :"supervisor",
"web recruiter manager" :"manager",
"content designer" :"designer",
"workshop attendee" :"attendee",
"NSF person interested in attending user interviews" :"user",
"NSF member of the team" :"member",
"VisualDesigner" :"designer",
"author" :"author",
"potential research participant" :"researcher",
"camp administrator" :"admin",
"parent" :"parent",
"camp worker" :"worker",
"camp counselor" :"counselor",
"moderator" :"moderator",
"estimator" :"estimator",
"participant" :"participant",
"Mike" :"visitor",
"Visitor" :"visitor",
"Admin" :"admin",
"Consumer" :"consumer",
"consumer" :"consumer",
"Data Analyst" :"analyst",
"Web Developer" :"developer",
"collection curator" :"curator",
"repository manager" :"manager",
"faculty member" :"member",
"DPC" :"curator",
"repoadmin" :"admin",
"DigitalRecords Archivist" :"archivist",
"batch submitter" :"submitter",
"manager" :"manager",
"data contributor" :"analyst",
"depositor" :"depositor",
"collection owner" :"owner",
"non-Duke researcher" :"researcher",
"app developer" :"developer",
"dataset developer" :"developer",
"data scientist" :"analyst",
"hydrator user" :"user",
"plugin developer" :"developer",
"pipeline developer" :"developer",
"pipeline designer" :"designer",
"system administrator" :"admin",
"release engineer" :"engineer",
"investigator" :"investigator",
"MRI operator" :"operator",
"lab administrator" :"admin",
"systems administrator" :"admin",
"supervisor" :"supervisor",
"administrator" :"admin",
"lab member" :"member",
"researcher" :"researcher",
"user" :"user",
"JuniorResearcher" :"researcher",
"OlderPerson" :"visitor",
"MedicalCaregiver" :"worker",
"SocialCaregiver" :"worker",
"InformalCaregiver" :"worker",
"serious games developer" :"developer",
"anonymoususer" :"user",
"trainingcoordinator" :"coordinator",
"attendee" :"attendee",
"trainee" :"trainee",
"authenticateduser" :"user",
"sponsor" :"sponsor",
"summit coordinator" :"coordinator",
"PI" :"visitor",
"data manager" :"manager",
"repository owner" :"owner",
"IT manager" :"manager",
"IT staff member" :"staff",
"legalofficer" :"officer",
"data librarian" :"analyst",
"archivemanager" :"manager",
"research head" :"researcher",
"funder" :"funder",
"metadata manager" :"manager",
"rector" :"rector",
"ethics manager" :"manager",
"repository operator" :"operator",
"IT officer" :"officer",
"Research Centre Director" :"researcher",
"Postgraduate Convenor" :"convenor",
"research support officer" :"researcher",
"Director of Research Integrity" :"researcher",
"archive" :"archivist",
"research evaluation manager" :"manager",
"institutional data steward" :"analyst",
"institutional data manager" :"manager",
"faculty data steward" :"analyst",
"Application Administrator" :"admin",
"Archivist" :"archivist",
"User" :"user",
"migration specialist" :"specialist",
"Authenticated User" :"user",
"Application Administrator" :"admin",
"Administrator" :"admin",
"archivist" :"archivist",
"Repository Manager" :"manager",
"Basic Data Entry user" :"user",
"ReadOnly user" :"user",
"System Administrator" :"admin",
"data reuser" :"user",
"externalcollaborator" :"collaborator",
"externalcoordinator" :"coordinator",
"research facility manager" :"manager",
"Bath Data Archive administrator" :"admin",
"Research Information manager" :"manager",
"UnivITservice" :"worker",
"academicpublisher" :"publisher",
"fundingbody" :"stakeholder",
"DAMS manager" :"manager",
"assistant archivist" :"archivist",
"donor representative" :"representative",
"researcher  I want to access digitized archival materials without mediation  so that I can decide what" :"researcher",
"Cornell faculty member" :"member",
"patron" :"patron",
"DB/IR administrator" :"admin",
"library staff member" :"staff",
"crossdeptprojteammember" :"member",
"Extension Administrator" :"admin",
"repository support team member" :"member",
"academicresearcher" :"researcher",
"externaluser" :"user",
"student" :"student",
"teachingfacultymember" :"member",
"Extension Administrator or Staff Content Manager" :"admin",
"Extension Educator" :"educator",
"librarydiscoveryoperator" :"operator",
"asset manager" :"manager",
"data curator" :"curator",
"Zooniverse admin" :"admin",
"Repository manager" : "manager"}

In [29]:
role_dict[df['role'][40]]

'developer'

In [30]:
# Making similar roles as one role
df['urole']=df['role'].apply(lambda x:role_dict[x])

In [31]:
df.sample(2)

,story,role,urole
436,As an NSF administrator I want to shadow the CMS implementation so that I can learn as the pro...,NSF administrator,admin
801,As a user I want to navigate forwards and backwards between log books and log book pages withou...,user,user


In [32]:
df['role']=df['urole']

In [33]:
df=df.drop('urole',axis=1)

In [34]:
df.sample(2)

,story,role
1571,As a researcher I want to promote/share my work through interactive maps so that I can display...,researcher
409,As a company I want to join the Scrum Alliance by paying a corporate membership fee so that I ...,company


In [35]:
df['role'].unique()

array(['user', 'designer', 'developer', 'engineer', 'owner', 'member',
       'tester', 'applicant', 'Customer', 'staff', 'supervisor',
       'inspector', 'admin', 'executive', 'employee', 'company',
       'representative', 'researcher', 'publisher', 'analyst', 'manager',
       'activist', 'trainer', 'visitor', 'editor', 'recruiter',
       'advertiser', 'sponsor', 'stakeholder', 'attendee', 'author',
       'parent', 'worker', 'counselor', 'moderator', 'estimator',
       'participant', 'consumer', 'curator', 'archivist', 'submitter',
       'depositor', 'investigator', 'operator', 'coordinator', 'trainee',
       'officer', 'funder', 'rector', 'convenor', 'specialist',
       'collaborator', 'patron', 'student', 'educator'], dtype=object)

In [36]:
# Now we want to assign the story points for each story based on heuristic approach
# First, We will decide min and max story points for a particular role then we assign a particular number 
# between the given range for a role
min_dict = {"user" : "3",
"designer" : "5",
"developer" : "6",
"engineer" : "4",
"owner" : "5",
"member" : "2",
"tester" : "4",
"applicant" : "1",
"Customer" : "3",
"staff" : "4",
"supervisor" : "4",
"inspector" : "3",
"admin" : "12",
"executive" : "5",
"employee" : "2",
"company" : "3",
"representative" : "1",
"researcher" : "15",
"publisher" : "1",
"analyst" : "25",
"manager" : "4",
"activist" : "1",
"trainer" : "12",
"visitor" : "1",
"editor" : "2",
"recruiter" : "1",
"advertiser" : "4",
"sponsor" : "1",
"stakeholder" : "1",
"attendee" : "5",
"author" : "16",
"parent" : "1",
"worker" : "8",
"counselor" : "2",
"moderator" : "4",
"estimator" : "4",
"participant" : "1",
"consumer" : "1",
"curator" : "4",
"archivist" : "10",
"submitter" : "1",
"depositor" : "1",
"investigator" : "5",
"operator" : "4",
"coordinator" : "7",
"trainee" : "4",
"officer" : "1",
"funder" : "1",
"rector" : "1",
"convenor" : "5",
"specialist" : "20",
"collaborator" : "15",
"patron" : "1",
"student" : "1",
"educator" : "5"}


In [37]:
max_dict = {"user" : "10",
"designer" : "12",
"developer" : "15",
"engineer" : "10",
"owner" : "13",
"member" : "7",
"tester" : "8",
"applicant" : "3",
"Customer" : "15",
"staff" : "12",
"supervisor" : "7",
"inspector" : "20",
"admin" : "23",
"executive" : "13",
"employee" : "7",
"company" : "7",
"representative" : "3",
"researcher" : "25",
"publisher" : "4",
"analyst" : "60",
"manager" : "13",
"activist" : "6",
"trainer" : "15",
"visitor" : "2",
"editor" : "7",
"recruiter" : "3",
"advertiser" : "9",
"sponsor" : "3",
"stakeholder" : "2",
"attendee" : "15",
"author" : "27",
"parent" : "2",
"worker" : "14",
"counselor" : "5",
"moderator" : "9",
"estimator" : "10",
"participant" : "9",
"consumer" : "6",
"curator" : "7",
"archivist" : "23",
"submitter" : "3",
"depositor" : "2",
"investigator" : "18",
"operator" : "9",
"coordinator" : "15",
"trainee" : "15",
"officer" : "3",
"funder" : "3",
"rector" : "4",
"convenor" : "6",
"specialist" : "30",
"collaborator" : "26",
"patron" : "3",
"student" : "4",
"educator" : "9"}

In [38]:
# Try to assign value of story point between min and max range
import random
df['story_point']=df['role'].apply(lambda x: random.randint(int(min_dict[x]),int(max_dict[x])))

In [39]:
df

,story,role,story_point
0,As a Data user I want to have the 12-19-2017 deletions processed.,user,9
1,As a UI designer I want to redesign the Resources page so that it matches the new Broker desig...,designer,5
2,As a UI designer I want to report to the Agencies about user testing so that they are aware of...,designer,8
3,As a UI designer I want to move on to round 2 of DABS or FABS landing page edits so that I can...,designer,5
4,As a UI designer I want to move on to round 2 of Homepage edits so that I can get approvals fr...,designer,8
...,...,...,...
1666,As a Zooniverse admin I want to know when Zoonibot should give an explanation.,admin,17
1667,As a Zooniverse admin I want to know what Zoonibot should say to a volunteer.,admin,22
1668,As a Zooniverse admin I want to group subjects by similarity.,admin,15
1669,As a Zooniverse admin I want to recommend different projects to volunteers based on their previ...,admin,13


In [40]:
# Pickle the data for further use
import pickle


In [41]:
f=open('first_df.pkl','wb')
pickle.dump(df,f)
f.close()

In [42]:
# Pickling check:
g=open('first_df.pkl','rb')
pdf=pickle.load(g)
g.close()

In [43]:
pdf

,story,role,story_point
0,As a Data user I want to have the 12-19-2017 deletions processed.,user,9
1,As a UI designer I want to redesign the Resources page so that it matches the new Broker desig...,designer,5
2,As a UI designer I want to report to the Agencies about user testing so that they are aware of...,designer,8
3,As a UI designer I want to move on to round 2 of DABS or FABS landing page edits so that I can...,designer,5
4,As a UI designer I want to move on to round 2 of Homepage edits so that I can get approvals fr...,designer,8
...,...,...,...
1666,As a Zooniverse admin I want to know when Zoonibot should give an explanation.,admin,17
1667,As a Zooniverse admin I want to know what Zoonibot should say to a volunteer.,admin,22
1668,As a Zooniverse admin I want to group subjects by similarity.,admin,15
1669,As a Zooniverse admin I want to recommend different projects to volunteers based on their previ...,admin,13


In [44]:
# Fin